In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from typing import Literal
from shapely.geometry import Point

from s2shores.bathy_debug.spatial_correlation_bathy_estimator_debug import SpatialCorrelationBathyEstimatorDebug
from s2shores.waves_exceptions import WavesEstimationError

from utils import initialize_sequential_run, build_dataset

In [ ]:
base_path = Path("/work/LITTORAL/MPData-Notebooks").resolve()
test_case: Literal["7_4", "8_2"] = "8_2"
method: Literal["spatial_corr", "spatial_dft", "temporal_corr"] = "spatial_corr"

product_path: Path = base_path / "products" / f"SWASH_{test_case}/testcase_{test_case}.tif"
config_path: Path = base_path / f"reference_results/debug_pointswash_{method}/wave_bathy_inversion_config.yaml"
debug_file: Path = base_path / f"debug_points/debug_points_SWASH_{test_case}.yaml"

estimation_point = Point(451.0, 499.0)

In [ ]:
bathy_estimator, ortho_bathy_estimator, ortho_sequence, config = initialize_sequential_run(
    product_path=product_path,
    config_path=config_path,
    point=estimation_point,
)

If you want to change any parameter of the configuration, modify the values of the object `config` as you would with a dict.  

Example:
```python
config["parameter"] = "new_value"
```

In [ ]:
self = SpatialCorrelationBathyEstimatorDebug(
    estimation_point,
    ortho_sequence,
    bathy_estimator,
)

if not self.can_estimate_bathy():
    raise WavesEstimationError("Cannot estimate bathy.")

## Preprocess images

Modified attributes:
- local_estimator.ortho_sequence.\<elements\>.pixels

In [ ]:
self.preprocess_images()

## Find direction

Modified attributes:
- None

New variables:
- estimated_direction

In [ ]:
estimated_direction = self.find_direction()

## Compute radon transforms

New elements:
- local_estimator.randon_transforms

In [ ]:
self.compute_radon_transforms(estimated_direction)

## Compute spatial correlation

New elements:
- local_estimator.sinograms

New variables:
- correlation_signal

In [ ]:
correlation_signal = self.compute_spatial_correlation(estimated_direction)

## Compute wavelength

Modified attributes:
- None

New variables:
- wavelength

In [ ]:
config["GLOBAL_ESTIMATOR"]["MIN_T"] = 5
# config["GLOBAL_ESTIMATOR"]["MIN_T"] = 10

In [ ]:
wavelength = self.compute_wavelength(correlation_signal)
wavelength

## Compute delta position

Modified attributes:
- None

New variables:
- delta_position

In [ ]:
delta_position = self.compute_delta_position(correlation_signal, wavelength)

## Save wave field estimation

New elements:
- local_estimator.bathymetry_estimations

In [ ]:
self.save_wave_field_estimation(estimated_direction, wavelength, delta_position)

In [ ]:
self.bathymetry_estimations[0].is_physical()

In [ ]:
dataset = build_dataset(
    bathy_estimator,
    ortho_bathy_estimator,
    self,
)
dataset

## TODO

- Créer une fonction pour les premières étapes ✅
- Documenter les modifications des attributs d'instance de chaque méthode du run ✅
- Eclater la cellule de run ✅
- rentrer dans chaque étape ✅
- flag _DEFAULT pour chaque étape ou custom function ✅
- faire la même chose pour les 3 méthodes
- vérifier la VT ✅
- vérifier l'exécution sur JupyterHub


Option: Configuration pydantic ✅